__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-16_15-10_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

5515


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

5515


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
3949,1026615293,"Участок 6,5 сот. (ИЖС)",5800000,,"садоводческое товарищество Сатурн, аллея Защитников 35-й Батареи","Продается участок 6,5 сот. На Алее Защитников 35 батареи, в первой линии Казачинского шоссе. Участок расположен в Казачьей бухте, рядом 35 батарея, ровный, имеет прямоугольную форму. Получено Гпзу, строить можно — жилой дом в высоту 8 м.! \n\nИмеются сети —",14,2022-09-15 10:31:26.039,sevastopol,6.50,True,5.80,0.89,4-8
5356,2247568188,Участок 10 сот. (ИЖС),11000000,,ул. Абазы,"Продаю свой учacтoк Ижс в районe 5 км., \npaспoложeнный нa улицe Aбазы, 47! \nУчасток paспoложен рядoм c пpиродным пapкoм Mакcимoвa Дaчa. \n\nCoceди пo учaстку уже пoстpoились и прoживают. \nЧистый воздуx! На участке рacтут cocны. В 3 мин xодьбы достраиваются ш",20,2022-09-16 15:10:34.059,sevastopol,10.00,True,11.00,1.10,8-20
3533,2447720155,Участок 8 сот. (ИЖС),4800000,,"с. Андреевка, Усадебная ул., 19","Продам земельный участок в живописном тихом уголке Крыма, г. Севастополь \n\nУчасток юридически чистый есть полный комплект документов, готов к строительству, подведено электричество.\n\n с. Андреевка, с развитой инфраструктурой , есть все что может потребоват",5,2022-09-15 10:31:26.039,sevastopol,8.00,True,4.80,0.60,4-8


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5515 entries, 0 to 5514
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            5515 non-null   int64         
 1   title               5515 non-null   object        
 2   price               5515 non-null   int64         
 3   obj_name            5515 non-null   object        
 4   adr                 5515 non-null   object        
 5   description         5515 non-null   object        
 6   avito_page          5515 non-null   int64         
 7   ts                  5515 non-null   datetime64[ns]
 8   place               5515 non-null   object        
 9   area                5515 non-null   float64       
 10  is_IJS              5515 non-null   bool          
 11  priceM              5515 non-null   float64       
 12  priceMU             5515 non-null   float64       
 13  area_size_category  5515 non-null   category    

---

In [14]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

5515


### обновляем таблицу адресов

In [15]:
from lib.locator import AddressTransformer

data['adr'] = AddressTransformer().transform( data['adr'] )

places = { # поле 'place' и название города
#'bahchisaray': 'Бахчисарай',
#      'sudak': 'Судак',
 'sevastopol': 'Севастополь,',
}

for k,v in places.items():
    print(k,v) # докидываем в строку адреса название города если его нет
    f =  (data['place']==k) & (~data['adr'].str.match(v))
    data.loc[f,'adr'] = v +' '+ data.loc[f,'adr']

sevastopol Севастополь,


---

In [16]:
from lib.locator import LocationUpdater
# from lib.locator import AddressTransformerSev
from lib.locator import GeocoderYandex
from yandex_keys import keys


loc_file_path = 'data/location.pkl'# таблица адресов

loc = LocationUpdater(
        locator=GeocoderYandex(keys[2]),
        # address_transformer=AddressTransformerSev(),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-16 15:33:31 | LocationUpdater: 10044 addresses in location table
[INFO    ] 2022-09-16 15:33:31 | LocationUpdater: 10055 addresses total
[INFO    ] 2022-09-16 15:33:31 | LocationUpdater: 10044 addresses defined
[INFO    ] 2022-09-16 15:33:31 | LocationUpdater: 11 addresses undefined


  0%|          | 0/11 [00:00<?, ?it/s]

[INFO    ] 2022-09-16 15:33:34 | LocationUpdater: 11 new addresses found


In [17]:
loc.sample(3)

,adr,latitude,longitude,truncated
9881,"Севастополь, район Нахимовский, Победа, проспект Победы, 59 к3",44.58,33.57,NaN
2497,"Севастополь, район Гагаринский, Омега, улица Челнокова, 14/1",44.59,33.43,False
7205,территория ТСН Рыбак-7,44.57,33.44,False


In [18]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

### дополняем данные геометкой

In [19]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 5529
записей без геометки: 0


In [20]:
assert len(data)>0
data.to_pickle(result_file_path)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5529 entries, 0 to 5528
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            5529 non-null   int64         
 1   title               5529 non-null   object        
 2   price               5529 non-null   int64         
 3   obj_name            5529 non-null   object        
 4   adr                 5529 non-null   object        
 5   description         5529 non-null   object        
 6   avito_page          5529 non-null   int64         
 7   ts                  5529 non-null   datetime64[ns]
 8   place               5529 non-null   object        
 9   area                5529 non-null   float64       
 10  is_IJS              5529 non-null   bool          
 11  priceM              5529 non-null   float64       
 12  priceMU             5529 non-null   float64       
 13  area_size_category  5529 non-null   category    